In [2]:
# logistic Regression 

import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

pickle_file = 'housingNantesAllInfoFullImages.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_prices_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_prices_labels']

  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)

image_size = train_dataset.shape[1]

('Training set', (1882, 56, 56), (1882, 1))
('Validation set', (764, 56, 56), (764, 1))


In [3]:
# clean prices == 0

price_equals_zero_list = []

for i,price in enumerate(train_labels):
    if price == 0:
        price_equals_zero_list.append(i)  

print len(price_equals_zero_list)   

new_dim = train_labels.shape[0]-len(price_equals_zero_list)

train_labels_no_zeros = np.zeros((new_dim, 1))
train_images_no_zeros = np.zeros((new_dim, image_size,image_size))

j = 0
for i,price in enumerate(train_labels):
    if price != 0:
        train_labels_no_zeros[j][0] = price  
        j += 1

j = 0
for i,image in enumerate(train_dataset):
    if i not in price_equals_zero_list:
        train_images_no_zeros[j,:,:] = image  
        j += 1

        
        
# for validation:

price_equals_zero_list = []

for i,price in enumerate(valid_labels):
    if price == 0:
        price_equals_zero_list.append(i)  

print len(price_equals_zero_list)   

new_dim = valid_labels.shape[0]-len(price_equals_zero_list) 

valid_labels_no_zeros = np.zeros((new_dim, 1))
valid_images_no_zeros = np.zeros((new_dim, image_size,image_size))

j = 0
for i,price in enumerate(valid_labels):
    if price != 0:
        valid_labels_no_zeros[j][0] = price  
        j += 1

j = 0
for i,image in enumerate(valid_dataset):
    if i not in price_equals_zero_list:
        valid_images_no_zeros[j,:,:] = image  
        j += 1        

16
18


In [4]:
train_labels = train_labels_no_zeros
valid_labels = valid_labels_no_zeros

In [5]:
# reformat 
train_images = train_images_no_zeros.reshape((-1, image_size * image_size)).astype(np.float32)
valid_images = valid_images_no_zeros.reshape((-1, image_size * image_size)).astype(np.float32)

num_labels= 1

dtype = np.float32

In [6]:

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation into constants that are
  # attached to the graph.
    tf_train_images = tf.placeholder(dtype, shape=[None,image_size * image_size])
    tf_train_price = tf.placeholder(dtype, shape=[None,train_labels.shape[1]])
    tf_valid_images = tf.placeholder(dtype, shape=[None,image_size * image_size])
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the loss as the difference between price estimated and real prices.
    
    price_estimation = tf.matmul(tf_train_images, weights) + biases
       
    loss = tf.reduce_mean(tf.abs(price_estimation - tf_train_price))
    
  # Optimizer.
  # We are going to find the minimum of this loss using AdagradOptimizer.
  #  AdagradOptimizer
    optimizer = tf.train.MomentumOptimizer(25, 0.001).minimize(loss)
  
  # Predictions for the training and validation.
    train_prediction = price_estimation
    valid_prediction = tf.matmul(tf_valid_images, weights) + biases

In [7]:
num_steps = 3001

def accuracy(prediction, price):
  return np.mean(100 - 100*np.abs((prediction - price) / price))

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions,valid_predictions = session.run([optimizer, loss, train_prediction,valid_prediction]
                                        , feed_dict={tf_train_images: train_images,tf_train_price: train_labels,tf_valid_images: valid_images })
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_predictions, valid_labels))

Initialized
Loss at step 0: 337470.375000
Training accuracy: 0.0%
Validation accuracy: 0.0%
Loss at step 100: 275536.343750
Training accuracy: 27.7%
Validation accuracy: 26.0%
Loss at step 200: 258123.156250
Training accuracy: 29.5%
Validation accuracy: 28.6%
Loss at step 300: 251011.000000
Training accuracy: 29.4%
Validation accuracy: 28.6%
Loss at step 400: 246665.031250
Training accuracy: 29.2%
Validation accuracy: 28.5%
Loss at step 500: 243655.203125
Training accuracy: 29.2%
Validation accuracy: 28.5%
Loss at step 600: 241538.562500
Training accuracy: 29.1%
Validation accuracy: 28.4%
Loss at step 700: 239812.515625
Training accuracy: 29.0%
Validation accuracy: 28.2%
Loss at step 800: 238375.671875
Training accuracy: 28.8%
Validation accuracy: 27.9%
Loss at step 900: 237115.359375
Training accuracy: 28.8%
Validation accuracy: 27.8%
Loss at step 1000: 235961.140625
Training accuracy: 28.8%
Validation accuracy: 27.8%
Loss at step 1100: 234912.015625
Training accuracy: 28.8%
Validatio